In [135]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn

import pandas as pd

from lmfit import minimize, Parameters

%matplotlib inline

In [235]:
data = pd.read_csv('resources/hst_color.csv', header=1, skiprows=[2], index_col=0,
                     skipinitialspace=True)

I need to clean up the data. I need to get the y-color (SDSS) and the x-color (HST) to not have `nan` and to get uncertainties. Also calculate uncertainties from the SNR and the magnitudes.

In [264]:
c = data['blue mag']/data['blueSNR'] + data['red mag']/data['redSNR']
data.insert(10, 'hst uncert', c)
cleanData = data.dropna()

,blueSNR,blueSource,blue mag,redSNR,redSource,red mag,color,g band,r band,sdss color,hst uncert
snid,,,,,,,,,,,
12781,9.388407,1.008574,NaN,15.883151,2.083933,NaN,NaN,24.067487,23.131326,0.936161,NaN
12860,20.733757,2.504576,18.554349,28.064364,3.924191,17.909909,0.644440,23.245719,22.530629,0.715090,1.533059
12874,3.886333,0.349395,NaN,1.547429,0.135448,NaN,NaN,24.814628,23.971412,0.843216,NaN
12928,13.704707,1.586626,NaN,23.494865,3.341392,NaN,NaN,23.914053,23.589620,0.324432,NaN
12930,7.090148,0.690027,NaN,9.864843,1.036041,NaN,NaN,24.198032,23.907693,0.290339,NaN
13038,5.545996,0.497140,NaN,-0.229203,-0.018449,NaN,NaN,25.154634,25.858991,-0.704357,NaN
13135,3.986819,0.369163,NaN,25.158194,3.403613,NaN,NaN,25.066907,24.081255,0.985652,NaN
13354,6.486538,0.688632,NaN,4.696899,0.446708,NaN,NaN,24.408977,23.648278,0.760699,NaN
13411,22.843225,2.653871,18.491485,29.844373,3.963552,17.899073,0.592412,23.919013,23.034183,0.884830,1.409242


In [83]:
def chi_sqr_linear(params, x, y, yUncert):
    parvals = params.valuesdict()
    m = parvals['slope']
    b = parvals['intercept']
    
    model = m*x + b
    return (model-y)/yUncert

In [266]:
params = Parameters()
params.add('slope', value=1.0)
params.add('intercept', value=0.0)

# x, y, yUncert = sdss, hst, hstUncert
x, y, yUncert = cleanData['sdss color'], cleanData['color'], cleanData['hst uncert']
out = minimize(chi_sqr_linear, params, args=(x, y, yUncert))

In [267]:
out.redchi
# out.chisqr
# out.var_names
# out.ndata
# out.params['slope']
# out.residual

0.036821947060988174

Sort data by smaller of two SNRs. Then calcualte the chi-squared (or something) starting with first 3 data points then adding one more data point at a time, calculating a new chi-squared (or something).

In [283]:
# c = data['blue mag']/data['blueSNR'] + data['red mag']/data['redSNR']
data[['blueSNR', 'redSNR']].min(axis=1)
data.insert(10, 'minSNR', data[['blueSNR', 'redSNR']].min(axis=1))
cleanData = data.dropna()

In [299]:
data.sort_values('minSNR', ascending=False, inplace=True)

In [303]:
data.iloc[:3]

,blueSNR,blueSource,blue mag,redSNR,redSource,red mag,color,g band,r band,sdss color,minSNR,hst uncert
snid,,,,,,,,,,,,
14284,97.416735,28.512205,15.913608,148.564252,63.398361,14.889089,1.024519,21.026248,20.128068,0.898180,97.416735,0.263576
14437,75.993186,18.192970,16.401425,123.390088,44.655579,15.269595,1.131831,22.760290,21.807200,0.953090,75.993186,0.339578
14279,73.934560,20.627485,16.265069,105.644565,39.466873,15.403702,0.861367,20.891803,20.068498,0.823305,73.934560,0.365800


In [320]:
params = Parameters()
params.add('slope', value=1.0)
params.add('intercept', value=0.0)

x = data.dropna().iloc[:3]['sdss color'].values
y = data.dropna().iloc[:3]['color'].values
yUncert = data.dropna().iloc[:3]['hst uncert'].values
# y, yUncert = cleanData['sdss color'], cleanData['color'], cleanData['hst uncert']
out = minimize(chi_sqr_linear, params, args=(x, y, yUncert))

In [321]:
out.params, out.chisqr

(Parameters([('slope',
              <Parameter 'slope', value=2.0880635863787695 +/- 0.0753, bounds=[-inf:inf]>),
             ('intercept',
              <Parameter 'intercept', value=-0.85468981728953541 +/- 0.0675, bounds=[-inf:inf]>)]),
 0.00038436896394908066)

In [315]:
params = Parameters()
params.add('slope', value=1.0)
params.add('intercept', value=0.0)

x = data.dropna().iloc[:10]['sdss color'].values
y = data.dropna().iloc[:10]['color'].values
yUncert = data.dropna().iloc[:10]['hst uncert'].values
out = minimize(chi_sqr_linear, params, args=(x, y, yUncert))
out.params

Parameters([('slope',
             <Parameter 'slope', value=0.91397076536359967 +/- 0.16, bounds=[-inf:inf]>),
            ('intercept',
             <Parameter 'intercept', value=0.12754283652110204 +/- 0.151, bounds=[-inf:inf]>)])

In [316]:
params = Parameters()
params.add('slope', value=1.0)
params.add('intercept', value=0.0)

x = data.dropna().iloc[:25]['sdss color'].values
y = data.dropna().iloc[:25]['color'].values
yUncert = data.dropna().iloc[:25]['hst uncert'].values
out = minimize(chi_sqr_linear, params, args=(x, y, yUncert))
out.params

Parameters([('slope',
             <Parameter 'slope', value=0.96456633706968486 +/- 0.099, bounds=[-inf:inf]>),
            ('intercept',
             <Parameter 'intercept', value=0.08442936823788022 +/- 0.0916, bounds=[-inf:inf]>)])

In [317]:
params = Parameters()
params.add('slope', value=1.0)
params.add('intercept', value=0.0)

x = data.dropna()['sdss color'].values
y = data.dropna()['color'].values
yUncert = data.dropna()['hst uncert'].values
out = minimize(chi_sqr_linear, params, args=(x, y, yUncert))
out.params

Parameters([('slope',
             <Parameter 'slope', value=0.96456633706968486 +/- 0.099, bounds=[-inf:inf]>),
            ('intercept',
             <Parameter 'intercept', value=0.08442936823788022 +/- 0.0916, bounds=[-inf:inf]>)])

In [319]:
out.chisqr

0.77326088828075168

In [331]:
for i in range(3, len(data.dropna())):
    params = Parameters()
    params.add('slope', value=1.0)
    params.add('intercept', value=0.0)

    x = data.dropna().iloc[:i]['sdss color'].values
    y = data.dropna().iloc[:i]['color'].values
    yUncert = data.dropna().iloc[:i]['hst uncert'].values
    out = minimize(chi_sqr_linear, params, args=(x, y, yUncert))
#     print(out.params['slope'], out.chisqr, out.redchi)
    print(out.chisqr, out.redchi)

0.000384368963949 0.000384368963949
0.186907161587 0.0934535807935
0.242760520256 0.0809201734188
0.242787636851 0.0606969092127
0.303027348547 0.0606054697094
0.337300136641 0.0562166894402
0.451631692764 0.064518813252
0.549566346661 0.0686957933326
0.595268270088 0.0661409188987
0.598471902015 0.0598471902015
0.617737860814 0.0561579873467
0.670268141802 0.0558556784835
0.672112501281 0.051700961637
0.675677260147 0.048262661439
0.678836938574 0.0452557959049
0.690090673314 0.0431306670821
0.692559897481 0.0407388174989
0.705139084064 0.0391743935591
0.766024364983 0.0403170718412
0.766025366933 0.0383012683467


something is wrong with the error or something. I should not have a chi-squared of 0.7?

In [329]:
range(3, len(data.dropna()))

range(3, 23)

In [ ]:
for i in range(3, len(data.dropna())):
    params = Parameters()
    params.add('slope', value=1.0)
    params.add('intercept', value=0.0)

    x = data.dropna().iloc[:i]['sdss color'].values
    y = data.dropna().iloc[:i]['color'].values
    yUncert = data.dropna().iloc[:i]['hst uncert'].values
    out = minimize(chi_sqr_linear, params, args=(x, y, yUncert))
#     print(out.params['slope'], out.chisqr, out.redchi)
    print(out.chisqr, out.redchi)